In [1]:
import os
from dotenv import load_dotenv

In [11]:
from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON

In [8]:
load_dotenv(".env")
api_key = os.environ["API_KEY"]

In [35]:
channel_ids = ['UCZJHUwR_ctkdoaHb0FeAhew',
             ]

In [34]:
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = build(
    api_service_name, api_version, developerKey=api_key)

In [25]:
def get_channel_stats(youtube, channel_ids):

    all_data = []
    
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id = ','.join(channel_ids)
    )
    response = request.execute()
    JSON(response)
    
    # loop through items
    for item in response['items']:
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
                'views': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount'],
                'playlistID': item['contentDetails']['relatedPlaylists']['uploads']
               }
        all_data.append(data)
    
    return(pd.DataFrame(all_data))

In [26]:
channel_stats = get_channel_stats(youtube, channel_ids)

In [32]:
channel_stats

,channelName,subscribers,views,totalVideos,playlistID
0,mistermister668,41900,16540179,1392,UUZJHUwR_ctkdoaHb0FeAhew


In [52]:
playlist_id = "OLAK5uy_lHH8ZP8T30cWlLUz7_EucPITqWT8hCRxc"

def get_video_ids(youtube, playlist_id):
    
    video_ids = []
    
    request = youtube.playlistItems().list(
        part="snippet, contentDetails",
        playlistId=playlist_id,
        maxResults=25
    )
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtube.playlistItems().list(
        part="snippet, contentDetails",
        playlistId=playlist_id,
        maxResults=25
        )
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])
        
        next_page_token = response.get('nextPageToken')
        
    return video_ids

In [53]:
video_ids = get_video_ids(youtube, playlist_id)

In [55]:
len(video_ids)

11

In [62]:
def get_video_details(youtube, video_ids):

    all_video_info = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet, contentDetails, statistics",
            id=','.join(video_ids[i:i+25])
        )

        response = request.execute()

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favoriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
            
        return pd.DataFrame(all_video_info)

In [63]:
video_df = get_video_details(youtube, video_ids)

In [64]:
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favoriteCount,commentCount,duration,caption
0,INi6kaG9CT4,John Scofield - Topic,Camelus,Provided to YouTube by Universal Music Group\n...,"[John Scofield, ジョン・スコフィールド, ジョンスコフィールド, Überj...",2019-01-09T10:23:39Z,39755,438,0,2,PT5M18S,false
1,T5GojQrgJxY,John Scofield - Topic,Boogie Stupid,Provided to YouTube by Universal Music Group\n...,"[John Scofield, ジョン・スコフィールド, ジョンスコフィールド, Überj...",2019-01-09T10:23:39Z,49925,645,0,1,PT5M6S,false
2,l7ISW29UZds,John Scofield - Topic,Endless Summer,Provided to YouTube by Universal Music Group\n...,"[John Scofield, ジョン・スコフィールド, ジョンスコフィールド, Überj...",2019-01-09T10:23:39Z,68167,1000,0,1,PT6M9S,false
3,jOljJ_jH4dE,John Scofield - Topic,Dub Dub,Provided to YouTube by Universal Music Group\n...,"[John Scofield, ジョン・スコフィールド, ジョンスコフィールド, Überj...",2019-01-09T10:23:39Z,24190,270,0,0,PT6M7S,false
4,wMFVQdkrero,John Scofield - Topic,Cracked Ice,Provided to YouTube by Universal Music Group\n...,"[John Scofield, ジョン・スコフィールド, ジョンスコフィールド, Überj...",2019-01-09T10:24:09Z,27534,488,0,0,PT5M54S,false
5,ZeknY7BvAtQ,John Scofield - Topic,Al Green Song,Provided to YouTube by Universal Music Group\n...,"[John Scofield, ジョン・スコフィールド, ジョンスコフィールド, Überj...",2019-01-09T10:23:55Z,19144,205,0,0,PT5M51S,false
6,DgMMQnmIY7s,John Scofield - Topic,Snake Dance,Provided to YouTube by Universal Music Group\n...,"[John Scofield, ジョン・スコフィールド, ジョンスコフィールド, Überj...",2019-01-09T10:23:39Z,19336,228,0,1,PT7M14S,false
7,kP08nvlZ11Y,John Scofield - Topic,Scotown,Provided to YouTube by Universal Music Group\n...,"[John Scofield, ジョン・スコフィールド, ジョンスコフィールド, Überj...",2019-01-09T10:23:39Z,21530,283,0,0,PT4M36S,false
8,vDKbDu8qzOQ,John Scofield - Topic,Torero,Provided to YouTube by Universal Music Group\n...,"[John Scofield, ジョン・スコフィールド, ジョンスコフィールド, Überj...",2019-01-09T10:23:39Z,10775,151,0,0,PT5M52S,false
9,6M8MlakToZE,John Scofield - Topic,Curtis Knew,Provided to YouTube by Universal Music Group\n...,"[John Scofield, ジョン・スコフィールド, ジョンスコフィールド, Überj...",2019-01-09T10:23:40Z,13495,169,0,1,PT4M45S,false
